### 1.Data Import

In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('S:/Codes/DataHackthon/Time Series Analysis Demand Forcast/train.csv')
test_df  = pd.read_csv('S:/Codes/DataHackthon/Time Series Analysis Demand Forcast/test.csv')
test_df['units_sold'] = -1

In [2]:
train_df.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,1,17/01/11,8091,216418,99.0375,111.8625,0,0,20
1,2,17/01/11,8091,216419,99.0375,99.0375,0,0,28
2,3,17/01/11,8091,216425,133.9500,133.9500,0,0,19
3,4,17/01/11,8091,216233,133.9500,133.9500,0,0,44
4,5,17/01/11,8091,217390,141.0750,141.0750,0,0,52


In [3]:
data = pd.concat([train_df,test_df])

In [4]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()                                              #store_id
le2 = LabelEncoder()                                             # sku_id
data['sku_id'] = le1.fit_transform(data['sku_id'])
data['store_id'] = le2.fit_transform(data['store_id'])

data.fillna(250,inplace = True)

###  2.Feature Generations

In [5]:
features = data.copy()

temp3 = data[['sku_id','store_id','base_price','total_price']].groupby(['sku_id','store_id']).mean().rename(columns = 
                {'total_price':'tot_grp_id','base_price':'base_grp_id'}).reset_index()

features = features.merge(right = temp3,on=['sku_id','store_id'])
features['base_grp_id_diff'] = features['base_price'] - features['base_grp_id']
features['tot_grp_id_diff'] = features['base_price'] - features['tot_grp_id']

In [6]:
data = data.merge(right = features[['record_ID','base_grp_id', 'tot_grp_id', 'base_grp_id_diff', 'tot_grp_id_diff']],on = 'record_ID')

data.drop(columns = ['record_ID'],inplace = True)

In [7]:
data.head()

,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,base_grp_id,tot_grp_id,base_grp_id_diff,tot_grp_id_diff
0,17/01/11,3,1,99.0375,111.8625,0,0,20,106.152465,102.780634,5.710035,9.081866
1,17/01/11,3,2,99.0375,99.0375,0,0,28,106.207658,103.126849,-7.170158,-4.089349
2,17/01/11,3,3,133.9500,133.9500,0,0,19,131.345863,126.870158,2.604137,7.079842
3,17/01/11,3,0,133.9500,133.9500,0,0,44,130.186796,126.413556,3.763204,7.536444
4,17/01/11,3,5,141.0750,141.0750,0,0,52,165.450528,154.627553,-24.375528,-13.552553


In [8]:
data.index = pd.to_datetime(data['week'])
data['week'] = data.index.week
data.reset_index(drop = True)

data['featured_cum_display'] = data['is_featured_sku']*data['is_display_sku']

data['price_percent'] = (data['base_price']-data['total_price'])/data['base_price']

In [9]:
data.head()

,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,base_grp_id,tot_grp_id,base_grp_id_diff,tot_grp_id_diff,featured_cum_display,price_percent
week,,,,,,,,,,,,,,
2011-01-17,3,3,1,99.0375,111.8625,0,0,20,106.152465,102.780634,5.710035,9.081866,0,0.11465
2011-01-17,3,3,2,99.0375,99.0375,0,0,28,106.207658,103.126849,-7.170158,-4.089349,0,0.00000
2011-01-17,3,3,3,133.9500,133.9500,0,0,19,131.345863,126.870158,2.604137,7.079842,0,0.00000
2011-01-17,3,3,0,133.9500,133.9500,0,0,44,130.186796,126.413556,3.763204,7.536444,0,0.00000
2011-01-17,3,3,5,141.0750,141.0750,0,0,52,165.450528,154.627553,-24.375528,-13.552553,0,0.00000


###  3.train_test_split

In [10]:
train = data.loc[data['units_sold'] > 0]
test = data.loc[data['units_sold'] == -1]

test.drop(columns = ['units_sold'],inplace = True)

X = train.drop(columns = ['units_sold'])
y = train['units_sold']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((112612, 13), (37538, 13), (112612,), (37538,))

###  4.Modelling

In [ ]:
from sklearn.ensemble import RandomForestRegressor
base_rfr = RandomForestRegressor(

    n_estimators = 1000,
    criterion = 'mse',
    n_jobs    = -1,
    verbose   = 5,
    
)

base_rfr.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 1000building tree 2 of 1000

building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s


building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000
building tree 42 of 1000
building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000
building tree 46 of 1000
building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000
building tree 51 of 1000
building tree 52 of 1000
building tree 53 of 1000
building tree 54 of 1000
building tree 55 of 1000
building tree 56 of 1000


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    7.8s


...

building tree 442 of 1000
building tree 443 of 1000
building tree 444 of 1000
building tree 445 of 1000
building tree 446 of 1000
building tree 447 of 1000
building tree 448 of 1000
building tree 449 of 1000
building tree 450 of 1000
building tree 451 of 1000
building tree 452 of 1000
building tree 453 of 1000
building tree 454 of 1000
building tree 455 of 1000
building tree 456 of 1000
building tree 457 of 1000
building tree 458 of 1000
building tree 459 of 1000
building tree 460 of 1000
building tree 461 of 1000
building tree 462 of 1000
building tree 463 of 1000
building tree 464 of 1000
building tree 465 of 1000
building tree 466 of 1000
building tree 467 of 1000
building tree 468 of 1000
building tree 469 of 1000
building tree 470 of 1000
building tree 471 of 1000
building tree 472 of 1000
building tree 473 of 1000
building tree 474 of 1000
building tree 475 of 1000
building tree 476 of 1000
building tree 477 of 1000
building tree 478 of 1000
building tree 479 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.5min


building tree 642 of 1000
building tree 643 of 1000
building tree 644 of 1000
building tree 645 of 1000
building tree 646 of 1000
building tree 647 of 1000
building tree 648 of 1000
building tree 649 of 1000
building tree 650 of 1000
building tree 651 of 1000
building tree 652 of 1000
building tree 653 of 1000
building tree 654 of 1000
building tree 655 of 1000
building tree 656 of 1000
building tree 657 of 1000
building tree 658 of 1000
building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000
building tree 663 of 1000
building tree 664 of 1000
building tree 665 of 1000
building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000
building tree 673 of 1000
building tree 674 of 1000
building tree 675 of 1000
building tree 676 of 1000
building tree 677 of 1000
building tree 678 of 1000
building tree 679 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  2.1min


building tree 875 of 1000
building tree 876 of 1000
building tree 877 of 1000
building tree 878 of 1000
building tree 879 of 1000
building tree 880 of 1000
building tree 881 of 1000
building tree 882 of 1000
building tree 883 of 1000
building tree 884 of 1000
building tree 885 of 1000


In [ ]:
from sklearn.metrics import mean_squared_log_error as msle
100 * np.sqrt(msle(y_test,base_rfr.predict(X_test)))

In [ ]:
sub = pd.DataFrame(test_df['record_ID'])
sub['units_sold'] = base_rfr.predict(test)
sub.to_csv('submission.csv')